In [2]:
from typing import Generator
from copy import deepcopy

In [3]:
class BinStr:
    def __init__(self, v="", *, lpad=0, rpad=0):
        if isinstance(v, str):
            if any(i not in {"", "0", "1"} for i in v):
                raise ValueError("Values must be binary (0/1)")
            self.v = "".rjust(lpad, "0") + v + "".ljust(rpad, "0")
        elif isinstance(v, Generator):
            self.v = "".join(map(str, v))

    def __repr__(self):
        return self.v

    def __len__(self):
        return len(self.v)

    def __iter__(self):
        yield from map(int, self.v)

    def __getitem__(self, x):
        return BinStr(self.v[x])

    def __reversed__(self):
        return BinStr(self.v[::-1])

    def __invert__(self):
        return BinStr(1 - a for a in self)

    def __or__(self, other):
        return BinStr(self.v + other.v)

    def __lshift__(self, other):
        temp, self.v = self[:other], repr(self[other:])
        return temp

    def __int__(self):
        return sum(int(repr(self[i])) * 2 ** (len(self) - i - 1) for i in range(len(self)))

    def __add__(self, other):
        if len(self) != len(other):
            if len(self) > len(other):
                other.v = other.v.rjust(len(self), "0")
            else:
                self.v = self.v.rjust(len(other), "0")

        carry = 0
        z = [None] * len(self)

        gen = (a + b + carry for a, b in zip(*map(reversed, [self, other])))

        for i, w in enumerate(gen, start=1):
            carry, z[-i] = divmod(w, 2)

        if carry:
            z.insert(0, 1)

        return BinStr(_ for _ in z)

    def __xor__(self, other):
        return BinStr(abs(a - b) for a, b in zip(self, other))

    def __truediv__(self, G):
        """
        101 11010100
            101
             111
             101
              100
              101
                110
                101
                 110
                 101
                  11
        """
        D = deepcopy(self)
        R = BinStr()  # CRC bits
        r = len(G) - 1

        offset = wnd_base = len(G)
        print(G, D)

        while D:
            R |= D << offset

            if not D and len(R) != len(G):
                break

            if wnd_base != len(G):
                print(" " * wnd_base, R)

            R ^= G
            R.v = R.v.lstrip("0")

            print(" " * wnd_base, G)
            print(" " * wnd_base, "-" * len(G))

            offset = len(G) - len(R)
            wnd_base += offset

        R = BinStr(R.v.rjust(len(G) - 1, "0"))
        print(" " * wnd_base, R)

        del D
        return R

In [4]:
D = BinStr("101110")
G = BinStr("1001")
r = len(G) - 1

R = (D | BinStr("0" * r)) / G
print()
(D | R) / G

1001 101110000
     1001
     ----
       1010
       1001
       ----
         1100
         1001
         ----
          1010
          1001
          ----
            011

1001 101110011
     1001
     ----
       1010
       1001
       ----
         1101
         1001
         ----
          1001
          1001
          ----
              000


000

In [5]:
def checksum(a, b):
    c = a + b
    if len(c) > len(a):
        c = c[1:] + BinStr("1")
    return ~c

In [6]:
def test(actual, expected):
    assert repr(actual) == expected

def test_all(actual, expected, op):
    assert all(op(i) == j for i, j in zip(actual, expected))

# Test [REVERSED]
test(reversed(BinStr("0011")), "1100")
# Test [NOT]
test(~BinStr("1"), "0")
# Test [ADD w/ Padding]
test(BinStr("00") + BinStr("1"), "01")
test(BinStr("1") + BinStr("00"), "01")
# Test [XOR]
test(BinStr("1010") ^ BinStr("0011"), "1001")

# Test [INT]
test_all(map(BinStr, ["0", "1", "10", "11", "100", "101", "110"]), range(6), int)

# Test [ADD] and [Checksum w/ Overflow]
test(checksum(BinStr("11010"), BinStr("11100")), "01000")
# Test [Checksum w/o Overflow]
test(checksum(BinStr("0"), BinStr("1")), "0")

In [7]:
a = BinStr("1111")
b = BinStr("1")

checksum(a, b)

1110